In [1]:
# import the necessary packages
# import os
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from sklearn.preprocessing import LabelBinarizer
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from imutils import paths
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os

Using TensorFlow backend.


In [2]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMG_DIMS = (224, 224, 3)

MODEL_NAME = "gender"

data_set_dir = 'data/dataset/' + MODEL_NAME

In [3]:
def split_train_test_set(directory):
    
    classifiers = next(os.walk(data_set_dir))[1]
    
    print("[INFO] loading images from {}".format(directory))
    # initialize the data and labels
    data = []
    labels = []
    
    # grab the image paths and randomly shuffle them
    image_paths = sorted(list(paths.list_images(directory)))
    random.seed(42)
    random.shuffle(image_paths)
    
    # loop over the input images
    for image_path in image_paths:
        # load the image, pre-process it, and store it in the data list
        image = cv2.imread(image_path)
        image = cv2.resize(image, (IMG_DIMS[0], IMG_DIMS[1]))
        image = img_to_array(image)
        data.append(image)
 
        # extract the class label from the image path and update the
        # labels list        
        label = image_path.split(os.path.sep)[-2]
        l = classifiers.index(label)
        labels.append(l)
        
    # scale the raw pixel intensities to the range [0, 1]
    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)
    
    print("[INFO] Binarizing the labels")
    # binarize the labels
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
     
    print("[INFO] Splitting the Train and Test sets")
    # partition the data into training and testing splits using 75% of
    # the data for training and the remaining 25% for testing
    train_X, test_X, train_Y, test_Y = train_test_split(data,
                                                        labels,
                                                        test_size=0.2,
                                                        random_state=42)

    # convert the labels from integers to vectors
    train_Y = to_categorical(train_Y, num_classes=2)
    test_Y = to_categorical(test_Y, num_classes=2)
    
    return train_X, test_X, train_Y, test_Y, lb

In [4]:
def show_model_performance_graph(model, epochs):
    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    N = epochs
    plt.plot(np.arange(0, N), 
             model.history["loss"], 
             label="train_loss")
    plt.plot(np.arange(0, N), 
             model.history["val_loss"], 
             label="val_loss")
    
    plt.plot(np.arange(0, N), 
             model.history["acc"], 
             label="train_acc")
    plt.plot(np.arange(0, N), 
             model.history["val_acc"], 
             label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.show()

In [5]:
def train_model(dataset_dir):
    train_X, test_X, train_Y, test_Y, lb = split_train_test_set(dataset_dir)
    
    print("[INFO] Augmenting image data...")
    # construct the image generator for data augmentation
    aug = ImageDataGenerator(rotation_range=25, 
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode="nearest")
    
    # initialize the model
    print("[INFO] compiling model...")
    model = VGGFace.build(IMG_DIMS, 'vgg_face_weights.h5')
    opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt, 
                  metrics=["accuracy"])
    
    # train the network
    print("[INFO] training network...")
    H = model.fit_generator(
        aug.flow(train_X, train_Y, batch_size=BS),
        validation_data=(test_X, test_Y), 
        steps_per_epoch=len(train_X) // BS,
        epochs=EPOCHS, 
        verbose=1)
    # save the model to disk
    print("[INFO] serializing network...")
    model.save(MODEL_NAME + "_model.h5")
    print("[INFO] serializing label binarizer...")
    f = open(MODEL_NAME + '_labels', 'wb')
    f.write(pickle.dumps(lb))
    f.close()
    
    return H, model, lb


In [7]:
model = VGGFace.build(IMG_DIMS, 'vgg-face-keras.h5')

ValueError: Dimension 2 in both shapes must be equal, but are 3 and 64. Shapes are [3,3,3,64] and [3,3,64,3]. for 'Assign_27' (op: 'Assign') with input shapes: [3,3,3,64], [3,3,64,3].

In [ ]:
# show_model_performance_graph(h, EPOCHS)